COMPONENT : <b>`FINAL MODEL TESTING`</b>

In [ ]:
# Mounting Google Drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
# Importing necessary libraries

import io
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from PIL import Image

In [92]:
# Defining the VGG-13 architecture
'''
We have modified the architecture of VGG13 by adding Batch Normalization
after every convolutional layer
'''

class VGG13_with_BatchNorm(nn.Module):
    def __init__(self, num_classes=4):
        super(VGG13_with_BatchNorm, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [93]:
# Defining transforms for data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [94]:
# Loading the model state_dict
state_dict = torch.load('/content/drive/MyDrive/Colab Notebooks/MS - CSE 676 B - Deep Learning/vgg13_model_2_BEST_VGG_MODEL.pkl', map_location='cuda')

In [95]:
# Instantiating the model and loading the state_dict
model = VGG13_with_BatchNorm()
model.load_state_dict(state_dict)
model.eval()

VGG13_with_BatchNorm(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14)

In [96]:
# Function for pre-processing the image (even after "transform" cell)
def preprocess_image(image_path, transform):
    image = Image.open(image_path).convert('RGB')  # Ensuring image is in RGB mode
    image = transform(image).unsqueeze(0)  # Adding batch dimension
    return image

In [97]:
def convert_to_rgb(image):
    # Repeating the grayscale image across all channels to create a pseudo-RGB image
    rgb_image = torch.cat([image] * 3, dim=1)
    return rgb_image

In [104]:
# Defining the path to the image
image_path = '/content/Te_0048.jpg'

In [105]:
# Preprocessing the image
image = preprocess_image(image_path, transform)

In [106]:
# Checking if the image is grayscale, if yes, we will convert it RGB
if image.shape[1] != 3:
    image = convert_to_rgb(image)

In [107]:
# Passing the preprocessed image through the model
with torch.no_grad():
    outputs = model(image)

In [108]:
# Getting the predicted class
_, predicted = torch.max(outputs, 1)

In [111]:
# Printing the predicted class

if(predicted.item() == 1):
  print("Predicted Brain Tumour : GLIOMA")
elif(predicted.item() == 2):
  print("Predicted Brain Tumour : MENINGIOMA")
elif(predicted.item() == 3):
  print("Predicted Brain Tumour : NO-TUMOUR")
elif(predicted.item() == 4):
  print("Predicted Brain Tumour : PITUITARY")

Predicted Brain Tumour : MENINGIOMA
